<a href="https://colab.research.google.com/github/LeeSeungwon89/Machine-learning_Theory/blob/master/5-2%20%EA%B5%90%EC%B0%A8%20%EA%B2%80%EC%A6%9D%EA%B3%BC%20%EA%B7%B8%EB%A6%AC%EB%93%9C%20%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **교차 검증과 그리드 서치**

## **데이터 준비**

데이터를 준비하겠습니다. 전 챕터에서 사용했던 와인 데이터셋을 사용하겠습니다.

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
print(wine)
print()
print(wine.info())

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

      alcohol  sugar    pH  class
0         9.4    1.9  3.51    0.0
1         9.8    2.6  3.20    0.0
2         9.8    2.3  3.26    0.0
3         9.8    1.9  3.16    0.0
4         9.4    1.9  3.51    0.0
...       ...    ...   ...    ...
6492     11.2    1.6  3.27    1.0
6493      9.6    8.0  3.15    1.0
6494      9.4    1.2  2.99    1.0
6495     12.8    1.1  3.34    1.0
6496     11.8    0.8  3.26    1.0

[6497 rows x 4 columns]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB
None


In [2]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)
print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


## **검증 세트**

현재까지는 훈련 세트로 모델을 훈련하고 테스트 세트로 모델을 평가했습니다. 그러나 테스트 세트만으로 계속 성능을 확인하다 보면 테스트 세트에 일반화 성능을 점점 맞추게 되는 셈입니다. 테스트 세트로 일반화 성능을 올바르게 예측하려면 모델을 만들고 나서 마지막에 딱 한 번만 사용하는 것이 좋습니다.

테스트 세트를 사용하지 않고 측정하는 방법은 훈련 세트를 다시 나누는 것입니다. 훈련 세트에서 다시 나눈 데이터를 **검증 세트(validation set)** 혹은 **개발 세트(dev set)**라고 부릅니다. 검증 세트를 활용하는 과정을 요약하면 아래와 같습니다.

1. 훈련 세트를 80%, 테스트 세트 20%로 나눕니다. 그리고 훈련 세트에서 다시 20%를 떼서 검증 세트로 만듭니다. 보통 테스트 세트와 검증 세트를 20 ~ 30%만큼 떼지만 훈련 세트가 아주 많다면 몇 %만 떼어 놓아도 전체 데이터를 대표하는 데 문제없습니다.

1. 훈련 세트로 모델을 훈련하고, 검증 세트로 모델을 평가합니다. 테스트하고 싶은 매개변수를 바꾸면서 가장 좋은 모델을 고르고, 최종적으로 결정한 매개변수를 사용하여 훈련 세트와 검증 세트를 합친 전체 훈련 데이터로 다시 모델을 훈련합니다.

1. 마지막에 테스트 세트로 최종 점수를 평가합니다.

위 방법은 실무에서 많이 사용하는 방법입니다.

### **검증 세트 만들기**

훈련 세트 샘플 5197개를 나눠서 훈련 세트 80%, 검증 세트 20%를 만들겠습니다. `train_test_split()` 메서드에 훈련 세트인 `train_input`과 `train_target`을 넣어서 훈련 세트를 `sub_input`과 `sub_target`으로 만들고, 검증 세트를 `val_input`, `val_target`으로 만듭니다.

In [5]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)
print(f'훈련 세트 크기: {sub_input.shape}')
print(f'검증 세트 크기: {val_input.shape}')

훈련 세트 크기: (4157, 3)
검증 세트 크기: (1040, 3)


### **결정 트리 모델 만들기**

훈련 세트와 검증 세트로 결정 트리 모델을 훈련하겠습니다.

In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(f'훈련 세트 점수: {dt.score(sub_input, sub_target)}')
print(f'검증 세트 점수: {dt.score(val_input, val_target)}')

훈련 세트 점수: 0.9971133028626413
검증 세트 점수: 0.864423076923077


과대적합이므로 매개변수를 바꿔서 더 좋은 모델을 찾아야 합니다.

## **교차 검증**

검증 세트를 만드느라 훈련 세트의 총량이 줄었습니다. 보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어지지만 그렇다고 검증 세트를 너무 조금 떼면 검증 점수가 불안정할 것입니다. 이 경우에 **교차 검증(cross validation)**을 활용하면 안정적인 검증 점수도 얻고 훈련에 더 많은 데이터도 사용할 수 있습니다.

교차 검증은 검증 세트를 떼어 평가하는 과정을 여러 번 반복합니다. 이렇게 얻은 점수들을 평균하여 최종 검증 점수를 얻습니다. **k-폴드(겹) 교차 검증(k-fold cross validation)**으로 이 방법을 실현할 수 있습니다. 

k-폴드(겹) 교차 검증(k-fold cross validation)은 훈련 세트를 k만큼 부분으로 나눠서 교차 검증을 수행하는 방법입니다. 보통 5나 10으로 나누는 5-폴드 교차 검증과 10-폴드 교차 검증을 많이 사용합니다. 이렇게 k만큼 부분으로 나눠서 교차 검증을 수행하면 총 데이터의 80 ~ 90%까지 훈련에 사용할 수 있습니다. 물론 검증 세트 자체는 줄지만 각 폴드에서 계산한 검증 점수를 평균하므로 최종 도출한 검증 점수는 안정적이라고 볼 수 있습니다.

3-폴드 교차 검증을 예로 들어 설명하겠습니다. 아래처럼 훈련 세트를 세 부분으로 나누고, 훈련 세트로는 모델을 훈련하며, 검증 세트로는 모델을 평가합니다.

* 훈련 세트, 훈련 세트, 검증 세트
 
* 훈련 세트, 검증 세트, 훈련 세트
 
* 검증 세트, 훈련 세트, 훈련 세트

총 3번만큼 모델을 평가하여 검증 점수를 도출하는 것입니다. `sklearn.model_selection` 패키지의 `cross_validate()` 메서드를 사용하여 교차 검증을 수행하겠습니다. 아래는 `cross_validate()` 메서드가 가진 매개변수 목록입니다.

- `scoring`: 검증에 사용할 평가 지표를 지정하는 매개변수입니다. 기본값은 분류 모델의 경우 정확도를 의미하는 `accuracy`, 회귀 모델의 경우 결정계수를 의미하는 `r2`입니다.

- `cv`: 교차 검증 폴드 수나 스플리터 객체를 지정하는 매개변수입니다. 기본값은 `5`입니다. 자세한 설명은 아래에서 이어가겠습니다.

- `n_jobs`: 교차 검증을 수행할 때 사용할 CPU 코어 수를 지정하는 매개변수입니다. 기본값은 `1`이며, `-1`로 지정하면 모든 시스템 코어를 사용합니다.

- `return_train_score`: 훈련 세트의 점수도 반환할지 정하는 매개변수입니다. 기본값은 `False`이며, `True`로 지정하면 훈련 세트 점수도 반환합니다.

`cross_validate()` 메서드를 사용하여 교차 검증을 수행하겠습니다. 이 메서드에 평가할 모델 객체를 첫 번째 매개변수에 전달하고, 직접 검증 세트를 떼지 않고 전체 훈련 세트를 두 세 번째 매개변수에 전달합니다. `cv` 매개변수는 따로 지정하지 않으므로 5-폴드 교차 검증입니다.

In [7]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00806403, 0.00719404, 0.00753808, 0.00733423, 0.00743771]), 'score_time': array([0.0008769 , 0.00072694, 0.00074053, 0.00075293, 0.00076175]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


`fit_time`, `score_time`, `test_score` 키를 가진 딕셔너리를 반환했습니다. `fit_time`은 훈련하는 시간, `score_time`은 검증하는 시간, `test_score`는 검증 폴드의 점수입니다.

### **최상의 검증 점수 확인하기**

`test_score`의 점수 5개를 평균하여 교차 검증의 최종 점수를 얻을 수 있습니다.

In [8]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


### **분할기로 폴드 나누기**

교차 검증을 수행하기 전에 위에서 `train_test_split()` 메서드로 전체 데이터를 섞었지만, 본래 `cross_validate()` 메서드는 훈련 세트를 섞어서 폴드를 나누지 않습니다. 교차 검증을 수행할 때 훈련 세트를 섞으려면 `cv` 매개변수에 **분할기(splitter)**를 지정해야 합니다. 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정합니다. 분류 모델일 경우 `StratifiedKFold()` 클래스를 지정하고, 회귀 모델일 경우 `KFold()` 클래스를 지정합니다.

바로 위에서 수행한 교차 검증은 아래처럼 표현할 수 있습니다.

In [ ]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


훈련 세트를 섞은 후 10-폴드 교차 검증을 수행해보겠습니다. 먼저 `StratifiedKfold()` 클래스 생성기로 인스턴스를 생성하고, 생성한 인스턴스를 `cross_validate()` 메서드의 `cv` 매개변수에 지정하겠습니다. 참고로 `KFold` 클래스도 동일한 방식으로 사용합니다.

`StratifiedKfold()` 클래스가 가진 매개변수는 아래와 같습니다.

- `n_splits`: 훈련 세트를 몇 부분으로 나눌지 결정하는 매개변수입니다. 몇(k) 폴드 교차 검증을 할지 결정합니다. 기본값은 `5`입니다.

- `shuffle`: 세트의 샘플을 섞을지 결정하는 매개변수입니다. 기본값은 `False`입니다. `True`로 지정하면 세트의 샘플을 섞습니다.

In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


5-폴드와 10-폴드의 점수 차이가 크지 않습니다.

## **하이퍼파라미터 튜닝**

모델이 학습하는 파라미터는 **모델 파라미터(Model parameter)**라고 부릅니다. 반면에 **하이퍼파라미터(Hyper parameter)**는 모델이 학습할 수 없어서 사용자가 직접 지정해야 하는 파라미터입니다. 다시 말하면 사용자가 직접 지정하는 클래스나 메서드의 매개변수를 의미합니다.

하이퍼파라미터 튜닝은 먼저 라이브러리가 제공하는 기본값으로 훈련하고, 매개변수를 조금씩 바꿔가며 모델을 훈련하면서 교차 검증을 수행하는 방식을 취합니다. 참고로 **AutoML**는 사용자의 개입 없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술입니다.

결정 트리 모델에서 `max_depth` 매개변수의 최적값을 찾았다고 가정하겠습니다. 이 상황에서 `max_depth` 값을 고정한 상태로 `min_samples_split` 매개변수의 최적값을 찾아가는 과정은 옳지 않습니다. `max_depth` 매개변수의 최적값은 `min_samples_split` 매개변수의 값이 바뀌면 함께 달라집니다. 두 매개변수의 값을 동시에 바꾸면서 최적값을 찾아야 합니다. 다만 매개변수가 많을수록 최적값을 일일이 찾기는 어렵습니다. 이런 상황에서는 **그리드 서치(Grid Search)**를 수행하는 `GridSearchCV` 클래스를 활용할 수 있습니다.

### **그리드 서치**

`GridSearchCV` 클래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행하는 클래스입니다. 별도로 `cross_validate()` 함수를 호출할 필요가 없습니다. 

아래는 `GridSearchCV` 클래스가 가진 매개변수 목록입니다. 교차 검증을 수행하는 `cross_validate()` 메서드와 동일한 매개변수를 가지고 있습니다.

- `scoring`: 검증에 사용할 평가 지표를 지정하는 매개변수입니다. 기본값은 분류 모델의 경우 정확도를 의미하는 `accuracy`, 회귀 모델의 경우 결정계수를 의미하는 `r2`입니다.

- `cv`: 교차 검증 폴드 수나 스플리터 객체를 지정하는 매개변수입니다. 기본값은 `5`입니다. 자세한 설명은 아래에서 이어가겠습니다.

- `n_jobs`: 교차 검증을 수행할 때 사용할 CPU 코어 수를 지정하는 매개변수입니다. 기본값은 `1`이며, `-1`로 지정하면 모든 시스템 코어를 사용합니다.

- `return_train_score`: 훈련 세트의 점수도 반환할지 정하는 매개변수입니다. 기본값은 `False`이며, `True`로 지정하면 훈련 세트 점수도 반환합니다.

#### **그리드 서치로 매개변수 최적값 찾기**

`GridSearchCV` 클래스로 `DecisionTreeClassifier` 클래스의 매개변수인 `min_impurity_decrease`의 최적값을 찾겠습니다. 참고로 `min_impurity_decrease` 매개변수는 노드를 분할하기 위한 불순도 감소 최소량을 지정하는 매개변수로 트리 가지치기에 사용합니다. 지정한 값보다 작으면 더이상 분할하지 않습니다.

먼저 `min_impurity_decrease` 매개변수를 0.0001 ~ 0.0005까지 리스트로 값을 부여하여 딕셔너리로 묶고 `params`라는 변수로 선언하겠습니다. 하이퍼파라미터 객체를 만드는 작업입니다.

In [14]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

`GridSearchCV` 클래스 생성기에 탐색 대상 모델과 변수를 전달하고, `gs` 인스턴스를 선언하겠습니다. 

참고로 `GridSearchCV` 클래스의 `cv` 매개변수는 폴드 수를 지정하는 매개변수입니다. 기본값은 5이므로 지정한 매개변수 `min_impurity_decrease`의 값마다 5-폴드 교차 검증을 수행합니다. 즉 $5 \times 5 = 25$개만큼 모델을 훈련합니다. `n_job` 매개변수는 병렬 실행에 사용할 CPU 코어 수를 지정하는 매개변수입니다. 기본값은 `1`입니다. `-1`로 지정하면 모든 코어를 사용합니다.

In [15]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

`gs` 인스턴스를 훈련하기 위해 `fit()` 메서드를 호출하면 `min_impurity_decrease` 값을 바꿔가며 5번 실행합니다. 훈련을 수행하겠습니다.

참고로 교차 검증에서 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 하지만, 그리드 서치는 훈련이 끝나면 모델 25개 중에 검증 점수가 가장 높은 모델의 매개변수 조합을 활용하여 전체 훈련 세트에서 자동으로 다시 모델을 훈련합니다. 이 모델은 `best_estimator_` 속성에 저장되고, 매개변수 최적값은 `best_params_` 속성에 저장됩니다. 모두 확인해보겠습니다.

In [16]:
gs.fit(train_input, train_target)

print(gs.best_estimator_)
print(gs.best_params_)

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)
{'min_impurity_decrease': 0.0001}


`DecisionTreeClassifier` 클래스에 매개변수의 최적값이 자동으로 입력됐습니다. `min_impurity_decrease` 매개변수의 최적값은 0.0001을 도출했습니다.

점수를 출력하려면 `gs` 인스턴스의 `best_estimator_` 속성을 `dt` 객체로 선언하고, 선언한 `dt` 객체에 `score()` 메서드를 적용합니다.

In [17]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


각 매개변수에서 수행한 교차 검증의 평균 점수는 `cv_results_` 속성의 `mean_test_score` 키에 저장됩니다. 아울러 교차 검증의 평균 점수 순위는 `cv_results_` 속성의 `rank_test_score` 키에 저장됩니다.

In [22]:
print(gs.cv_results_)
print()
print(f"교차 검증의 평균 점수: {gs.cv_results_['mean_test_score']}")
print(f"교차 검증의 평균 점수 순위: {gs.cv_results_['rank_test_score']}")

{'mean_fit_time': array([0.01200886, 0.00915728, 0.01068292, 0.00739532, 0.00710158]), 'std_fit_time': array([0.00303647, 0.00134618, 0.00156988, 0.0007407 , 0.00105177]), 'mean_score_time': array([0.00119801, 0.00107613, 0.00108914, 0.00095897, 0.00081038]), 'std_score_time': array([1.48006552e-04, 1.76647645e-04, 2.58710082e-04, 6.14107022e-05,
       9.18756443e-05]), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}], 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]), 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]), 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216,

첫 번째 점수가 가장 큽니다. 넘파이의 `argmax()` 메서드를 사용하면 가장 큰 값의 인덱스를 추출할 수 있습니다. 이 인덱스를 사용하여 매개변수 최적값인 `gs.best_params_` 속성을 동일하게 출력할 수 있습니다.

In [25]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(f'교차 검증의 평균 점수 중 가장 높은 점수의 위치: {best_index}')
print(f"사용한 모든 파라미터: {gs.cv_results_['params']}")
print(f"매개변수 최적값: {gs.cv_results_['params'][best_index]}")

교차 검증의 평균 점수 중 가장 높은 점수의 위치: 0
사용한 모든 파라미터: [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}]
매개변수 최적값: {'min_impurity_decrease': 0.0001}


#### **더 복잡한 매개변수 조합의 최적값 찾기**

그리드 서치로 `min_impurity_decrease`, `max_depth`, `min_samples_split` 매개변수 조합을 찾아보겠습니다. 매개변수의 최적값을 찾기 위해 `params` 변수에 딕셔너리로 된 조합을 전달하겠습니다. 매개변수 값의 간격은 임의로 지정하겠습니다.

참고로 `min_impurity_decrease` 매개변수는 노드를 분할하기 위한 불순도 감소 최소량을 지정하는 매개변수, `max_depth` 매개변수는 트리 깊이를 지정하는 매개변수, `min_samples_split` 매개변수는 노드를 나누기 위한 최소 샘플 수를 지정하는 매개변수입니다. 

In [26]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)
        }

첫 번째 매개변수에 지정한 배열은 9개, 두 번째 매개변수에 지정한 배열은 15개, 세 번째 매개변수에 지정한 배열은 10개입니다. 이 매개변수로 수행할 교차 검증 횟수는 $9 \times 15 \times 10 = 1,350$회이며, `cv` 매개변수의 기본값인 `5`로 5-폴드 교차 검증을 수행하므로 모델은 $1,350 \times 5 = 6,750$개가 만들어집니다.

그리드 서치를 수행하겠습니다.

In [27]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})


최상의 매개변수 조합을 확인하겠습니다.

In [28]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


최상의 교차 검증 점수를 확인하겠습니다.

In [29]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### **랜덤 서치**

**랜덤 서치(Random Search)**는 그리드 서치의 대안으로 사용할 수 있는 방법입니다. 기실 그리드 서치를 사용할 때 매개변수의 간격을 더 좁거나 넓은 간격으로 시도할 수도 있습니다. 그러나 매개변수 값이 수치일 때는 값의 범위나 간격을 미리 정하기 어려울 수도 있고, 너무 많은 매개변수 조합이 있으면 그리드 서치를 수행하는 시간이 오래 소요될 수도 있습니다. 이런 상황에서 랜덤 서치는 훌륭한 대안이 될 수 있습니다. 그리드 서치보다 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색할 수 있습니다.

랜덤 서치를 구현한 클래스는 `RandomizedSearchCV`입니다. 교차 검증으로 랜덤한 하이퍼파라미터를 탐색하는 클래스입니다. 최상의 모델을 찾은 후 훈련 세트 전체를 사용하여 최종 모델을 훈련합니다. 아래는 `RandomizedSearchCV` 클래스가 가진 매개변수입입니다. 교차 검증을 수행하는 `cross_validate()` 메서드와 동일한 매개변수를 가지고 있습니다.

- `scoring`: 검증에 사용할 평가 지표를 지정하는 매개변수입니다. 기본값은 분류 모델의 경우 정확도를 의미하는 `accuracy`, 회귀 모델의 경우 결정계수를 의미하는 `r2`입니다.

- `cv`: 교차 검증 폴드 수나 스플리터 객체를 지정하는 매개변수입니다. 기본값은 `5`입니다. 자세한 설명은 아래에서 이어가겠습니다.

- `n_jobs`: 교차 검증을 수행할 때 사용할 CPU 코어 수를 지정하는 매개변수입니다. 기본값은 `1`이며, `-1`로 지정하면 모든 시스템 코어를 사용합니다.

- `return_train_score`: 훈련 세트의 점수도 반환할지 정하는 매개변수입니다. 기본값은 `False`이며, `True`로 지정하면 훈련 세트 점수도 반환합니다.

랜덤 서치에는 매개변수 값의 목록을 전달하지 않고 매개변수를 샘플링 할 수 있는 확률 분포 객체를 전달합니다. 이 객체를 만들려면 **싸이파이(Scipy)**라는 수치 계산 전용 라이브러리가 가진 확률 분포 클래스를 사용해야 합니다.

예시를 들고자 주어진 범위에서 고르게 값을 뽑는 확률 분포 클래스 2개를 호출하겠습니다.

In [43]:
from scipy.stats import randint, uniform

`randint` 클래스와 `uniform` 클래스는 주어진 범위에서 고르게 값을 뽑는(균등 분포에서 샘플링하는) 클래스입니다. 난수 발생기와 유사합니다. `randint`는 정수값을 뽑고, `uniform` 클래스는 실수값을 뽑습니다. 랜덤 서치에 사용할 때 샘플링 횟수는 시스템 자원이 허락하는 범위 내에서 최대한 크게 하는 것이 좋습니다.

먼저 `randint` 클래스를 호출해보겠습니다. 클래스 생성기로 인스턴스를 생성하고, 생성한 인스턴스로 `rvs()` 메서드를 호출하겠습니다. `rvs()` 메서드는 랜덤 표본 생성(random variable sampling) 횟수를 지정하는 메서드입니다.

In [47]:
rgen = randint(0, 10)
print(rgen.rvs(10))

[1 4 4 1 8 2 1 7 1 9]


0~9 범위의 정수를 10개만큼 랜덤으로 샘플링했습니다.

정수를 1000개만큼 랜덤으로 샘플링하고 각 정수의 개수를 세겠습니다. 넘파이의 `unique()` 메서드가 가진 `return_counts` 매개변수를 `True`로 지정하면 개수도 출력합니다.

In [50]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([107,  91,  90,  96, 106,  91, 101,  93, 114, 111]))

두 배열은 같은 인덱스에 대응하여 각각 정수와 개수를 나타냅니다.

이번에는 `uniform` 클래스를 호출하겠습니다.

In [51]:
ugen = uniform(0, 1)
print(ugen.rvs(10))

[0.32075534 0.75082697 0.25328334 0.02869636 0.89800716 0.21841607
 0.92427481 0.83603033 0.61224837 0.89789375]


In [53]:
np.unique(ugen.rvs(10), return_counts=True)

(array([0.06739276, 0.14129709, 0.22440564, 0.25096548, 0.34052595,
        0.34367711, 0.53395619, 0.54055495, 0.75406934, 0.77824267]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

실수는 여러 값이 샘플링되므로 개수도 1개씩입니다.

#### **탐색할 매개변수의 딕셔너리 만들기**

탐색할 매개변수 값의 범위를 `randint` 클래스와 `uniform` 클래스를 사용하여 딕셔너리로 만들겠습니다. 매개변수 값의 범위는 임의로 지정하겠습니다. 아울러 `min_samples_leaf` 매개변수를 탐색 대상에 추가하겠습니다. `min_samples_leaf` 매개변수는 리프 노드가 되기 위한 최소 샘플의 개수를 지정하는 매개변수입니다. 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않습니다. 

In [54]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : randint(2, 25),
          'min_samples_leaf' : randint(1, 25)
        }

#### **샘플링 횟수 지정하기**

샘플링 횟수를 지정하겠습니다. 샘플링 횟수는 `RandomizedSearchCV` 클래스의 `n_iter` 매개변수에 지정합니다. 기본값은 `10`입니다.

In [57]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)

#### **랜덤 서치로 훈련하기**

`RandomizedSearchCV` 클래스로 훈련하고, 최상의 매개변수 조합과 교차 검증 점수를 확인하겠습니다. 

In [58]:
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


최적의 모델을 확인하고 테스트 세트의 점수를 확인하겠습니다.

In [59]:
dt = gs.best_estimator_
print(dt)
print(dt.score(test_input, test_target))

DecisionTreeClassifier(max_depth=39,
                       min_impurity_decrease=0.00034102546602601173,
                       min_samples_leaf=7, min_samples_split=13,
                       random_state=42)
0.86


테스트 세트에 대한 점수는 검증 세트에 대한 점수보다 일반적으로 조금 작습니다.

## **마무리**

`RandomizedSearchCV` 클래스를 사용할 때 `DecisionTreeClassifier` 클래스에 `splitter = random` 매개변수를 추가하고 다시 훈련해보겠습니다. `splitter` 매개변수의 기본값은 `best` 로 각 노드에서 최선의 분할을 찾습니다. `random`이면 무작위로 분할한 후 가장 좋은 것을 고릅니다. 다음 절에서 자세한 설명을 이어가겠습니다.

In [63]:
gs1 = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42),
                         params, n_iter=100, n_jobs=-1, random_state=42)
gs1.fit(train_input, train_target)

print(gs1.best_params_)
print(np.max(gs1.cv_results_['mean_test_score']))

dt = gs1.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077


검증 세트와 테스트 세트에서 성능이 둘 다 하락하여 과소적합이 되었습니다. 결정 트리 노드를 무작위로 분할했기 때문에 최적의 매개변수 조합을 찾지 못했습니다. 다만 샘플링 횟수를 늘리면 점수를 향상시킬 수 있습니다.

In [64]:
gs1 = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42),
                         params, n_iter=1000, n_jobs=-1, random_state=42)
gs1.fit(train_input, train_target)

print(gs1.best_params_)
print(np.max(gs1.cv_results_['mean_test_score']))

dt = gs1.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 46, 'min_impurity_decrease': 0.0001001883995565819, 'min_samples_leaf': 2, 'min_samples_split': 15}
0.8562636040571556
0.8415384615384616


점수가 향상됐습니다.